# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 27 2022 11:24AM,238267,10,SONSB0001802,"Nextsource Biotechnology, LLC",Released
1,May 27 2022 11:23AM,238264,10,7974011-BO,Eywa Pharma Inc.,Released
2,May 27 2022 11:23AM,238264,10,7995934-BO,Eywa Pharma Inc.,Released
3,May 27 2022 11:23AM,238264,10,8041698,Eywa Pharma Inc.,Released
4,May 27 2022 11:23AM,238264,10,8046556-BO,Eywa Pharma Inc.,Released
5,May 27 2022 11:23AM,238264,10,8067251,Eywa Pharma Inc.,Released
6,May 27 2022 11:23AM,238264,10,8067252,Eywa Pharma Inc.,Released
7,May 27 2022 11:23AM,238264,10,8067772,Eywa Pharma Inc.,Released
8,May 27 2022 11:23AM,238264,10,8069354,Eywa Pharma Inc.,Released
9,May 27 2022 11:23AM,238264,10,8069355,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,238263,Released,23
31,238264,Released,31
32,238265,Released,1
33,238266,Released,1
34,238267,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238263,NaN,NaN,NaN,23.0
238264,NaN,NaN,NaN,31.0
238265,NaN,NaN,NaN,1.0
238266,NaN,NaN,NaN,1.0
238267,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238159,0.0,0.0,0.0,1.0
238165,0.0,0.0,6.0,3.0
238168,0.0,0.0,0.0,1.0
238171,0.0,0.0,0.0,2.0
238179,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238159,0.0,0,0,1
238165,0.0,0,6,3
238168,0.0,0,0,1
238171,0.0,0,0,2
238179,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238159,0.0,0,0,1
1,238165,0.0,0,6,3
2,238168,0.0,0,0,1
3,238171,0.0,0,0,2
4,238179,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238159,0.0,,,1
1,238165,0.0,,6,3
2,238168,0.0,,,1
3,238171,0.0,,,2
4,238179,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 27 2022 11:24AM,238267,10,"Nextsource Biotechnology, LLC"
1,May 27 2022 11:23AM,238264,10,Eywa Pharma Inc.
32,May 27 2022 11:18AM,238266,20,Sartorius Stedim North America
33,May 27 2022 11:17AM,238265,10,ISDIN Corporation
34,May 27 2022 10:47AM,238263,19,"Innogenix, LLC"
57,May 27 2022 10:46AM,238262,19,"Innogenix, LLC"
58,May 27 2022 10:17AM,238257,19,"Bi-Coastal Pharma International, LLC"
59,May 27 2022 10:14AM,238256,17,Amatheon Pharmaceuticals
60,May 27 2022 10:13AM,238255,10,Amatheon Pharmaceuticals
61,May 27 2022 10:08AM,238254,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,May 27 2022 11:24AM,238267,10,"Nextsource Biotechnology, LLC",0.0,,,1
1,May 27 2022 11:23AM,238264,10,Eywa Pharma Inc.,0.0,,,31
2,May 27 2022 11:18AM,238266,20,Sartorius Stedim North America,0.0,,,1
3,May 27 2022 11:17AM,238265,10,ISDIN Corporation,0.0,,,1
4,May 27 2022 10:47AM,238263,19,"Innogenix, LLC",0.0,,,23
5,May 27 2022 10:46AM,238262,19,"Innogenix, LLC",0.0,,,1
6,May 27 2022 10:17AM,238257,19,"Bi-Coastal Pharma International, LLC",0.0,,,1
7,May 27 2022 10:14AM,238256,17,Amatheon Pharmaceuticals,0.0,,,1
8,May 27 2022 10:13AM,238255,10,Amatheon Pharmaceuticals,0.0,,,1
9,May 27 2022 10:08AM,238254,10,ISDIN Corporation,0.0,,,35


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 11:24AM,238267,10,"Nextsource Biotechnology, LLC",1,,
1,May 27 2022 11:23AM,238264,10,Eywa Pharma Inc.,31,,
2,May 27 2022 11:18AM,238266,20,Sartorius Stedim North America,1,,
3,May 27 2022 11:17AM,238265,10,ISDIN Corporation,1,,
4,May 27 2022 10:47AM,238263,19,"Innogenix, LLC",23,,
5,May 27 2022 10:46AM,238262,19,"Innogenix, LLC",1,,
6,May 27 2022 10:17AM,238257,19,"Bi-Coastal Pharma International, LLC",1,,
7,May 27 2022 10:14AM,238256,17,Amatheon Pharmaceuticals,1,,
8,May 27 2022 10:13AM,238255,10,Amatheon Pharmaceuticals,1,,
9,May 27 2022 10:08AM,238254,10,ISDIN Corporation,35,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 11:24AM,238267,10,"Nextsource Biotechnology, LLC",1,,
1,May 27 2022 11:23AM,238264,10,Eywa Pharma Inc.,31,,
2,May 27 2022 11:18AM,238266,20,Sartorius Stedim North America,1,,
3,May 27 2022 11:17AM,238265,10,ISDIN Corporation,1,,
4,May 27 2022 10:47AM,238263,19,"Innogenix, LLC",23,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 11:24AM,238267,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,May 27 2022 11:23AM,238264,10,Eywa Pharma Inc.,31.0,NaN,NaN
2,May 27 2022 11:18AM,238266,20,Sartorius Stedim North America,1.0,NaN,NaN
3,May 27 2022 11:17AM,238265,10,ISDIN Corporation,1.0,NaN,NaN
4,May 27 2022 10:47AM,238263,19,"Innogenix, LLC",23.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 11:24AM,238267,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,May 27 2022 11:23AM,238264,10,Eywa Pharma Inc.,31.0,0.0,0.0
2,May 27 2022 11:18AM,238266,20,Sartorius Stedim North America,1.0,0.0,0.0
3,May 27 2022 11:17AM,238265,10,ISDIN Corporation,1.0,0.0,0.0
4,May 27 2022 10:47AM,238263,19,"Innogenix, LLC",23.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3097000,122.0,18.0,1.0
12,238210,1.0,0.0,0.0
15,476473,2.0,5.0,0.0
16,952825,7.0,4.0,0.0
17,238256,1.0,0.0,0.0
18,238159,1.0,0.0,0.0
19,714782,25.0,0.0,0.0
20,1191106,35.0,8.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3097000,122.0,18.0,1.0
1,12,238210,1.0,0.0,0.0
2,15,476473,2.0,5.0,0.0
3,16,952825,7.0,4.0,0.0
4,17,238256,1.0,0.0,0.0
5,18,238159,1.0,0.0,0.0
6,19,714782,25.0,0.0,0.0
7,20,1191106,35.0,8.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,122.0,18.0,1.0
1,12,1.0,0.0,0.0
2,15,2.0,5.0,0.0
3,16,7.0,4.0,0.0
4,17,1.0,0.0,0.0
5,18,1.0,0.0,0.0
6,19,25.0,0.0,0.0
7,20,35.0,8.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,122.0
1,12,Released,1.0
2,15,Released,2.0
3,16,Released,7.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20
Status,,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,18.0,0.0,5.0,4.0,0.0,0.0,0.0,8.0
Released,122.0,1.0,2.0,7.0,1.0,1.0,25.0,35.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,5.0,4.0,0.0,0.0,0.0,8.0
2,Released,122.0,1.0,2.0,7.0,1.0,1.0,25.0,35.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,5.0,4.0,0.0,0.0,0.0,8.0
2,Released,122.0,1.0,2.0,7.0,1.0,1.0,25.0,35.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()